# 1. Data Cleaning & Engineering:

### Step 1: Data Cleaning and Feature Engineering

Imported the trip data with proper datetime parsing.
- Dropped any trips missing pickup/dropoff times.
- Converted key columns to numeric types for safe calculations.
- Removed duplicate entries to ensure data integrity.
- Added two new columns:
  - `trip_duration_minutes`: the duration of each trip in minutes.
  - `total_trip_cost`: the sum of all fare components (fare, extra, taxes, tip, tolls).
  Cleaned `trip_data.csv` by removing invalid or missing values.
- Computed new features: `trip_duration_minutes`, `total_trip_cost`.
- Cleaned `taxi_zone_geo.csv` and used it to enrich trip data with:
  - `pickup_zone`, `pickup_borough`
  - `dropoff_zone`, `dropoff_borough`
This will allow us to perform geographical and zone-based analysis.

- We retained null values in columns like `tip_amount`, `extra`, and `tolls_amount` because they reflect real-world cases such as unrecorded cash tips, non-applicable extras, or toll-free routes. Replacing them with zeros could introduce misleading assumptions. Instead, we preserved them to ensure accurate analysis and will handle them contextually during specific queries or modeling.

In [8]:
import pandas as pd
trip_df = pd.read_csv('/content/taxi_trip_data.csv', parse_dates=['pickup_datetime', 'dropoff_datetime'])

#Convert numeric columns
numeric_cols = ['passenger_count', 'trip_distance', 'fare_amount', 'extra',
                'mta_tax', 'tip_amount', 'tolls_amount', 'imp_surcharge',
                'pickup_location_id', 'dropoff_location_id']
trip_df[numeric_cols] = trip_df[numeric_cols].apply(pd.to_numeric, errors='coerce')

#Step 2: Clean Trip Data
trip_df = trip_df.dropna(subset=['pickup_datetime', 'dropoff_datetime', 'trip_distance', 'fare_amount'])
trip_df['trip_duration_minutes'] = (trip_df['dropoff_datetime'] - trip_df['pickup_datetime']).dt.total_seconds() / 60
trip_df = trip_df[(trip_df['trip_duration_minutes'] > 0) & (trip_df['trip_distance'] > 0)]
trip_df = trip_df[trip_df['fare_amount'] >= 0]
trip_df = trip_df[trip_df['passenger_count'] > 0]
trip_df = trip_df.drop_duplicates()
trip_df['total_trip_cost'] = trip_df[['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount']].sum(axis=1)

#Step 3: Clean Taxi Zone Geo CSV
zone_df = pd.read_csv('/content/taxi_zone_geo.csv')

# Drop empty geometry fields
zone_df = zone_df.dropna(subset=['zone_id', 'zone_name', 'borough'])

#Step 4: Merge Pickup Zone Info
zone_pickup = zone_df.rename(columns={
    'zone_id': 'pickup_location_id',
    'zone_name': 'pickup_zone',
    'borough': 'pickup_borough'
})
merged_df = trip_df.merge(zone_pickup[['pickup_location_id', 'pickup_zone', 'pickup_borough']], on='pickup_location_id', how='left')

#Step 5: Merge Dropoff Zone Info
zone_dropoff = zone_df.rename(columns={
    'zone_id': 'dropoff_location_id',
    'zone_name': 'dropoff_zone',
    'borough': 'dropoff_borough'
})
merged_df = merged_df.merge(zone_dropoff[['dropoff_location_id', 'dropoff_zone', 'dropoff_borough']], on='dropoff_location_id', how='left')

merged_df.head()


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code,store_and_fwd_flag,payment_type,fare_amount,extra,...,tolls_amount,imp_surcharge,pickup_location_id,dropoff_location_id,trip_duration_minutes,total_trip_cost,pickup_zone,pickup_borough,dropoff_zone,dropoff_borough
0,1,2018-05-11 17:40:00,2018-05-11 17:55:00,1,1.60,1,N,1,11.5,1.0,...,0.0,0.3,48,68,15.0,13.00,Clinton East,Manhattan,East Chelsea,Manhattan
1,2,2018-03-22 23:01:00,2018-03-22 23:25:00,1,9.52,1,N,1,28.5,0.5,...,0.0,0.3,138,230,24.0,35.46,LaGuardia Airport,Queens,Times Sq/Theatre District,Manhattan
2,2,2018-07-24 09:58:00,2018-07-24 10:22:00,1,2.17,1,N,1,15.5,0.0,...,0.0,0.3,234,48,24.0,17.50,Union Sq,Manhattan,Clinton East,Manhattan
3,2,2018-12-21 18:28:00,2018-12-21 18:35:00,1,0.86,1,N,2,6.0,1.0,...,0.0,0.3,79,125,7.0,7.50,East Village,Manhattan,Hudson Sq,Manhattan
4,1,2018-08-15 13:58:00,2018-08-15 14:05:00,1,0.30,1,N,2,5.5,0.0,...,0.0,0.3,233,233,7.0,6.00,UN/Turtle Bay South,Manhattan,UN/Turtle Bay South,Manhattan




---



# 2. Analytical Queries:

### Query (a): Most Common Payment Type per Time of Day

We used the pickup time to classify each trip into one of four time-of-day categories:
- Morning (05:00–11:59)
- Afternoon (12:00–17:59)
- Evening (18:00–21:59)
- Night (22:00–04:59)

We then grouped the trips by time of day and payment method (e.g., Credit Card, Cash) and found the most frequently used method for each time segment.

This analysis helps identify rider behavior patterns throughout the day.


In [11]:
#Helper function to classify time of day
def get_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 22:
        return 'Evening'
    else:
        return 'Night'

#Extract hour and classify time of day
merged_df['pickup_hour'] = merged_df['pickup_datetime'].dt.hour
merged_df['time_of_day'] = merged_df['pickup_hour'].apply(get_time_of_day)

#Map payment type codes to human-readable names
payment_map = {
    1: 'Credit Card',
    2: 'Cash',
    3: 'No Charge',
    4: 'Dispute',
    5: 'Unknown',
    6: 'Voided Trip'
}
merged_df['payment_label'] = merged_df['payment_type'].map(payment_map)

#Group by time of day and payment type, then count
payment_stats = (
    merged_df
    .groupby(['time_of_day', 'payment_label'])
    .size()
    .reset_index(name='count')
)

#Get most common payment type per time of day
most_common_payment = (
    payment_stats
    .sort_values('count', ascending=False)
    .drop_duplicates('time_of_day')
    .sort_values('time_of_day')  # Optional: sort output by day order
)

most_common_payment


,time_of_day,payment_label,count
1,Afternoon,Credit Card,219968
5,Evening,Credit Card,177164
9,Morning,Credit Card,186088
13,Night,Credit Card,133694


###  Query (b): Which boroughs generate the highest total revenue based on pickup locations, and how do they compare in terms of trip volume?

This analysis helps us understand which boroughs contribute the most to NYC taxi earnings.

Steps performed:
- Grouped the data by `pickup_borough`
- Calculated:
  - `total_revenue`: the sum of all trip costs per borough
  - `trip_count`: the total number of trips originating in each borough
- Sorted the results in descending order of total revenue

This gives us insight into high-activity and high-earning boroughs like Manhattan.


In [12]:
#Group by pickup borough and calculate total revenue and number of trips
borough_stats = (
    merged_df
    .groupby('pickup_borough')
    .agg(
        total_revenue=pd.NamedAgg(column='total_trip_cost', aggfunc='sum'),
        trip_count=pd.NamedAgg(column='pickup_borough', aggfunc='count')
    )
    .reset_index()
    .sort_values('total_revenue', ascending=False)
)

borough_stats


,pickup_borough,total_revenue,trip_count
3,Manhattan,13035289.62,936004
4,Queens,2881214.93,64297
1,Brooklyn,232060.02,12819
0,Bronx,28976.01,1132
2,EWR,1285.47,17
5,Staten Island,801.94,17


###Query (c): What is the average tip amount per passenger count?

We grouped trips by the number of passengers and calculated:
- `average_tip`: the mean of `tip_amount` per group
- `trip_count`: how many trips had that passenger count

This helps identify whether tipping behavior changes depending on group size (e.g., solo riders vs. groups).


In [14]:
#Group by passenger count and calculate average tip
avg_tip_by_passenger = (
    merged_df
    .groupby('passenger_count')
    .agg(
        average_tip=pd.NamedAgg(column='tip_amount', aggfunc='mean'),
        trip_count=pd.NamedAgg(column='tip_amount', aggfunc='count')
    )
    .reset_index()
    .sort_values('passenger_count')
)

avg_tip_by_passenger


,passenger_count,average_tip,trip_count
0,1,1.869362,738336
1,2,1.860966,152189
2,3,1.832186,43428
3,4,1.698026,20463
4,5,1.877342,46912
5,6,1.866375,28078
6,9,0.000000,1


### Query (d): What are the best 5 locations for drivers to pick up passengers from and at which time of the day?

To help drivers maximize their chances of getting passengers:
- We first identified the **top 5 pickup zones** with the highest total trip counts.
- Then, we analyzed the distribution of pickups in those zones across different **times of day** (Morning, Afternoon, Evening, Night).

This helps answer when and where it's best for a driver to be.


In [18]:
#Group by pickup zone and time of day, then count trips
pickup_stats = (
    merged_df
    .groupby(['pickup_zone', 'time_of_day'])
    .size()
    .reset_index(name='trip_count')
)

#Sort to get top 5 locations overall
top_pickup_zones = (
    pickup_stats
    .groupby('pickup_zone')['trip_count']
    .sum()
    .reset_index()
    .sort_values('trip_count', ascending=False)
    .head(5)['pickup_zone']
)

#Filter original stats to include only top 5 zones
top_pickup_stats = pickup_stats[pickup_stats['pickup_zone'].isin(top_pickup_zones)]

top_pickup_stats.sort_values(['pickup_zone', 'trip_count'], ascending=[True, False])


,pickup_zone,time_of_day,trip_count
536,Midtown Center,Afternoon,14463
537,Midtown Center,Evening,10605
538,Midtown Center,Morning,8199
539,Midtown Center,Night,6319
540,Midtown East,Afternoon,12043
541,Midtown East,Evening,10446
542,Midtown East,Morning,8859
543,Midtown East,Night,4913
772,Times Sq/Theatre District,Afternoon,9915
773,Times Sq/Theatre District,Evening,9271


### Query (e): What are the top 5 longest trips recorded in the dataset and display their corresponding trip duration, fare, pickup and dropoff zones, and payment type. Comment on whether these trips also resulted in high fares or if any anomalies are observed.

We first identified data anomalies where trip durations were exactly 1440 minutes (24 hours) — likely due to logging errors.  
To ensure reliability, we filtered out trips longer than 12 hours.

Then we selected the top 5 longest valid trips based on duration and displayed:
- Trip duration (in minutes)
- Fare amount
- Pickup and dropoff zones
- Payment method

This gives a more accurate insight into actual long-duration taxi usage in NYC.

In [20]:
#Add readable payment method again if not already added
payment_map = {
    1: 'Credit Card',
    2: 'Cash',
    3: 'No Charge',
    4: 'Dispute',
    5: 'Unknown',
    6: 'Voided Trip'
}
merged_df['payment_label'] = merged_df['payment_type'].map(payment_map)

#Get the top 5 longest trips by duration
longest_trips = (
    merged_df
    .sort_values('trip_duration_minutes', ascending=False)
    .head(5)[[
        'trip_duration_minutes',
        'fare_amount',
        'pickup_zone',
        'dropoff_zone',
        'payment_label'
    ]]
)

# Filter out trips longer than 12 hours (720 minutes)
merged_df = merged_df[merged_df['trip_duration_minutes'] < 720]

longest_trips

,trip_duration_minutes,fare_amount,pickup_zone,dropoff_zone,payment_label
239018,719.0,39.0,LaGuardia Airport,Times Sq/Theatre District,Credit Card
988008,718.0,38.0,LaGuardia Airport,Clinton East,Cash
659369,716.0,52.0,Midtown South,JFK Airport,Credit Card
547836,714.0,14.0,UN/Turtle Bay South,West Chelsea/Hudson Yards,Credit Card
247377,711.0,40.0,Times Sq/Theatre District,LaGuardia Airport,Credit Card


### Query (f): Which pickup and drop-off borough combinations represent the most frequent inter-borough travel flows? Present the top routes by trip count, and optionally include revenue per route.

**Steps performed:**
- Filtered only **inter-borough** trips where the pickup and dropoff boroughs are **not the same**.
- Grouped the dataset by `(pickup_borough, dropoff_borough)`.
- Calculated:
  - `trip_count`: the total number of trips for each borough pair.
  - `total_revenue`: the cumulative revenue generated by each route (optional insight).
- Sorted the results by `trip_count` to find the **top 10 inter-borough flows**.

**Key Insights:**
- The **Queens → Manhattan** route had the highest number of inter-borough trips, followed by **Manhattan → Queens**.
- **Manhattan → Brooklyn** is also a high-volume route.
- Routes to/from **airport-connected boroughs** (Queens, Manhattan, EWR) dominate the top revenue spots.
- While **Bronx and Staten Island** appear less frequently, their inclusion highlights low-volume, low-revenue segments.

This analysis helps understand cross-borough commuting patterns and revenue flows, valuable for transportation planning and taxi service optimization.

In [21]:
#Filter only inter-borough trips (where pickup and dropoff boroughs differ)
inter_borough_df = merged_df[merged_df['pickup_borough'] != merged_df['dropoff_borough']]

#Group by pickup-dropoff borough pair and aggregate trip count and revenue
inter_borough_stats = (
    inter_borough_df
    .groupby(['pickup_borough', 'dropoff_borough'])
    .agg(
        trip_count=pd.NamedAgg(column='fare_amount', aggfunc='count'),
        total_revenue=pd.NamedAgg(column='total_trip_cost', aggfunc='sum')
    )
    .reset_index()
    .sort_values('trip_count', ascending=False)
    .head(10)  # Show top 10 most common inter-borough routes
)

inter_borough_stats


,pickup_borough,dropoff_borough,trip_count,total_revenue
19,Queens,Manhattan,37582,1954207.24
14,Manhattan,Queens,31813,1337851.73
12,Manhattan,Brooklyn,26720,750741.33
17,Queens,Brooklyn,8839,406749.72
11,Manhattan,Bronx,4784,150770.66
7,Brooklyn,Manhattan,3809,94460.08
13,Manhattan,EWR,1854,178412.12
16,Queens,Bronx,1058,57179.88
8,Brooklyn,Queens,800,26675.21
2,Bronx,Manhattan,436,11606.08


# 3. SparkML Task: Trip Profiling: Predict Likelihood of High Tipping


### ML Setup: Define Target Variable and Select Features

To begin the machine learning task, we first define a binary target:
- `high_tip = 1` if the tip exceeds 15% of the fare
- `high_tip = 0` otherwise

Next, we selected a set of meaningful features likely to influence tipping behavior:
- `passenger_count`
- `trip_distance`
- `trip_duration_minutes`
- `fare_amount`
- `pickup_hour`

This prepares the dataset for model training in the next steps.

In [22]:
#Create the high_tip label
merged_df['high_tip'] = (merged_df['tip_amount'] > 0.15 * merged_df['fare_amount']).astype(int)

#Ensure pickup_hour column is present
if 'pickup_hour' not in merged_df.columns:
    merged_df['pickup_hour'] = merged_df['pickup_datetime'].dt.hour

#Select relevant features
feature_cols = [
    'passenger_count',
    'trip_distance',
    'trip_duration_minutes',
    'fare_amount',
    'pickup_hour'
]

#Final dataset for ML: X = features, y = label
X = merged_df[feature_cols]
y = merged_df['high_tip']


###ML Step: Train/Test Split and Classifier Training

We trained three classification models to predict whether a trip will receive a high tip:
- Logistic Regression
- Decision Tree
- Random Forest

Steps:
1. Split data (80% train, 20% test)
2. Scaled features using `StandardScaler`
3. Trained each model and evaluated performance using `precision`, `recall`, and `F1-score`

This allows us to compare model performance and choose the most effective one.

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

#Step 1: Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Step 2: Build models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier()
}

#Step 3: Train and evaluate each model
for name, model in models.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('classifier', model)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    print(f"\n {name} Results:")
    print(classification_report(y_test, y_pred))



 Logistic Regression Results:
              precision    recall  f1-score   support

           0       0.52      0.00      0.01     87725
           1       0.57      1.00      0.73    117694

    accuracy                           0.57    205419
   macro avg       0.55      0.50      0.37    205419
weighted avg       0.55      0.57      0.42    205419


 Decision Tree Results:
              precision    recall  f1-score   support

           0       0.44      0.43      0.43     87725
           1       0.58      0.60      0.59    117694

    accuracy                           0.53    205419
   macro avg       0.51      0.51      0.51    205419
weighted avg       0.52      0.53      0.53    205419


 Random Forest Results:
              precision    recall  f1-score   support

           0       0.45      0.37      0.41     87725
           1       0.59      0.67      0.62    117694

    accuracy                           0.54    205419
   macro avg       0.52      0.52      0.51    

We chose **Random Forest** because it achieved the best balance: **54% accuracy**, **0.62 F1-score** for high-tip trips, and reasonable recall for both classes. Unlike Logistic Regression (which had **57% accuracy** but **0% recall** for non-tippers), Random Forest handled class imbalance better and provided feature importance for interpretation.


### Feature Importance from Random Forest

We used the Random Forest model to extract feature importance scores, which tell us which variables most influenced the prediction of high tips.

This helps understand tipping behavior and could guide taxi policy or driver training (e.g., trip distance and fare size being strong indicators).


In [27]:
# Extract and train final Random Forest (without scaler, since it's tree-based)
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

# Get feature importances
importances = rf.feature_importances_
features = X.columns
importance_df = pd.DataFrame({
    'feature': features,
    'importance': importances
}).sort_values(by='importance', ascending=False)

# Show feature importance
importance_df


,feature,importance
1,trip_distance,0.588790
2,trip_duration_minutes,0.141146
3,fare_amount,0.117790
4,pickup_hour,0.111151
0,passenger_count,0.041123


The most important feature was `trip_distance` = 59%, followed by `trip_duration_minutes` and `fare_amount`. This suggests that longer and more expensive trips are more likely to receive higher tips. `pickup_hour` also had influence, indicating time-based tipping trends. `passenger_count` had the least impact.


**Final ML Summary:**

We built a machine learning model to predict whether a trip would result in a high tip (above 15% of the fare). After cleaning the data and engineering features, we trained three classifiers:

- **Logistic Regression** (Accuracy: 57%, but 0% recall for non-tippers)
- **Decision Tree** (Accuracy: 53%)
- **Random Forest** (Accuracy: 54%, F1-score: 0.62 for high-tippers)

We selected **Random Forest** as the final model due to its balanced performance and interpretability. Feature importance analysis revealed that:

- `trip_distance` was the most influential feature (≈ 59%)
- Followed by `trip_duration_minutes`, `fare_amount`, and `pickup_hour`
- `passenger_count` had minimal impact on tipping

These insights suggest that longer, more expensive trips taken at certain times of day are more likely to receive higher tips.
